In [2]:
import os
import json
import cv2
import random
import shutil

# Define dataset paths
json_path = "/home/idrone2/Desktop/rsm.json"  # Update this
images_dir = "/home/idrone2/Desktop/rsm_annotated"  # Update this
output_dir = "/home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2"

# Create train/val image and label folders
train_images_dir = os.path.join(output_dir, "images/train")
val_images_dir = os.path.join(output_dir, "images/val")
train_labels_dir = os.path.join(output_dir, "labels/train")
val_labels_dir = os.path.join(output_dir, "labels/val")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Load JSON file
with open(json_path, "r") as f:
    data = json.load(f)

# Get image filenames
image_filenames = list(data.keys())
random.shuffle(image_filenames)

# Split dataset (80% train, 20% val)
train_size = int(0.8 * len(image_filenames))
train_images = image_filenames[:train_size]
val_images = image_filenames[train_size:]

# Class mapping (Ensure correct order)
class_names = ["Red Spider Mite"]
class_mapping = {name: idx for idx, name in enumerate(class_names)}

# Function to normalize points
def normalize_points(points, img_width, img_height):
    return [(x / img_width, y / img_height) for x, y in points]

# Process each image annotation
for image_name in image_filenames:
    image_path = os.path.join(images_dir, image_name)
    img = cv2.imread(image_path)

    if img is None:
        continue

    img_height, img_width, _ = img.shape
    label_file_name = image_name.replace(".jpg", ".txt").replace(".JPG", ".txt")

    if image_name in train_images:
        shutil.copy(image_path, os.path.join(train_images_dir, image_name))
        label_file = os.path.join(train_labels_dir, label_file_name)
    else:
        shutil.copy(image_path, os.path.join(val_images_dir, image_name))
        label_file = os.path.join(val_labels_dir, label_file_name)

    with open(label_file, "w") as label_out:
        for region in data[image_name]["regions"].values():
            shape = region["shape_attributes"]
            if shape["name"] == "polygon":
                all_points_x = shape["all_points_x"]
                all_points_y = shape["all_points_y"]

                # Normalize polygon points
                normalized_polygon = normalize_points(
                    list(zip(all_points_x, all_points_y)), img_width, img_height
                )

                # Get class label
                class_label = region["region_attributes"]["label"]
                class_id = class_mapping[class_label]

                # Convert to YOLOv11 segmentation format: class_id x1 y1 x2 y2 ... xn yn
                polygon_str = " ".join([f"{x} {y}" for x, y in normalized_polygon])
                label_out.write(f"{class_id} {polygon_str}\n")

print("✅ Conversion to YOLOv11 segmentation format completed!")


✅ Conversion to YOLOv11 segmentation format completed!


[ WARN:0@100.385] global loadsave.cpp:268 findDecoder imread_('/home/idrone2/Desktop/rsm_annotated/annotations'): can't open/read file: check file path/integrity
[ WARN:0@100.385] global loadsave.cpp:268 findDecoder imread_('/home/idrone2/Desktop/rsm_annotated/categories'): can't open/read file: check file path/integrity
[ WARN:0@100.385] global loadsave.cpp:268 findDecoder imread_('/home/idrone2/Desktop/rsm_annotated/images'): can't open/read file: check file path/integrity
[ WARN:0@100.385] global loadsave.cpp:268 findDecoder imread_('/home/idrone2/Desktop/rsm_annotated/info'): can't open/read file: check file path/integrity


In [4]:
import os
import json
import cv2
import random
import shutil

# Define dataset paths
json_path = "/home/idrone2/Desktop/rsm.json"  # Update this
images_dir = "/home/idrone2/Desktop/rsm_annotated"  # Update this
output_dir = "/home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2"

# Create train/val image and label folders
train_images_dir = os.path.join(output_dir, "images/train")
val_images_dir = os.path.join(output_dir, "images/val")
train_labels_dir = os.path.join(output_dir, "labels/train")
val_labels_dir = os.path.join(output_dir, "labels/val")

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Load JSON file
with open(json_path, "r") as f:
    data = json.load(f)

# Extract image filenames from "images" section
images = {img["id"]: img["file_name"] for img in data["images"]}
random.shuffle(list(images.values()))

# Split dataset (80% train, 20% val)
train_size = int(0.8 * len(images))
train_images = list(images.values())[:train_size]
val_images = list(images.values())[train_size:]

# Class mapping (Ensure correct order)
class_names = ["Red Spider Mite"]
class_mapping = {name: idx for idx, name in enumerate(class_names)}

# Function to normalize points
def normalize_points(points, img_width, img_height):
    return [(x / img_width, y / img_height) for x, y in points]

# Process each image annotation
for annotation in data["annotations"]:
    image_id = annotation["image_id"]
    if image_id not in images:
        continue  # Skip if image ID not found in dataset

    image_name = images[image_id]
    image_path = os.path.join(images_dir, image_name)
    img = cv2.imread(image_path)

    if img is None:
        print(f"⚠️ Warning: Cannot open {image_path}")
        continue

    img_height, img_width = img.shape[:2]
    label_file_name = image_name.replace(".jpg", ".txt").replace(".JPG", ".txt")

    if image_name in train_images:
        shutil.copy(image_path, os.path.join(train_images_dir, image_name))
        label_file = os.path.join(train_labels_dir, label_file_name)
    else:
        shutil.copy(image_path, os.path.join(val_images_dir, image_name))
        label_file = os.path.join(val_labels_dir, label_file_name)

    with open(label_file, "w") as label_out:
        if "segmentation" in annotation:
            segmentation = annotation["segmentation"][0]  # COCO segmentation is a list of lists
            normalized_polygon = normalize_points(
                [(segmentation[i], segmentation[i+1]) for i in range(0, len(segmentation), 2)], img_width, img_height
            )

            # Get class label
            class_id = annotation["category_id"] - 1  # COCO format starts category IDs from 1

            # Convert to YOLO segmentation format: class_id x1 y1 x2 y2 ... xn yn
            polygon_str = " ".join([f"{x:.6f} {y:.6f}" for x, y in normalized_polygon])
            label_out.write(f"{class_id} {polygon_str}\n")

print("✅ Conversion to YOLOv11 segmentation format completed!")


✅ Conversion to YOLOv11 segmentation format completed!


In [5]:
from ultralytics import YOLO

# Load YOLOv11 model
model = YOLO("/home/idrone2/Desktop/Ranjith-works/yolo/yolo11s-seg.pt")  # Ensure you have the pre-trained weights

# Train the model
model.train(
    data="/home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2/data.yaml",
    epochs=50,
    imgsz=640,
    batch=16,
    device="cuda"  # Use "cpu" if no GPU is available
)


Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA RTX A2000 12GB, 11926MiB)
engine/trainer: task=segment, mode=train, model=/home/idrone2/Desktop/Ranjith-works/yolo/yolo11s-seg.pt, data=/home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=Fa

train: Scanning /home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2/labels/train... 28 images, 0 backgrounds, 0 corrupt: 100%|██████████| 28/28 [00:00<00:00, 2093.53it/s]

train: New cache created: /home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2/labels/train.cache



val: Scanning /home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2/labels/val... 7 images, 0 backgrounds, 0 corrupt: 100%|██████████| 7/7 [00:00<00:00, 731.92it/s]

val: New cache created: /home/idrone2/Desktop/Ranjith-works/yolo/yolo_dataset_2/labels/val.cache


...

                   all          7          7    0.00236      0.143    0.00214   0.000428    0.00236      0.143     0.0217    0.00477



50 epochs completed in 0.014 hours.
Optimizer stripped from runs/segment/train4/weights/last.pt, 20.5MB
Optimizer stripped from runs/segment/train4/weights/best.pt, 20.5MB

Validating runs/segment/train4/weights/best.pt...
Ultralytics 8.3.71 🚀 Python-3.10.12 torch-2.5.1+cu124 CUDA:0 (NVIDIA RTX A2000 12GB, 11926MiB)
YOLO11s-seg summary (fused): 265 layers, 10,067,203 parameters, 0 gradients, 35.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 26.23it/s]


                   all          7          7     0.0878      0.143     0.0324     0.0226      0.164      0.143     0.0407     0.0224
Speed: 0.1ms preprocess, 2.4ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to runs/segment/train4


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7133e1f9df60>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041, 